# Guided Project  - Profitable App Profiles for the App Store and Google Play Markets

**About**: For this project, we'll pretend we're working as data analysts for a company that builds Android and iOS mobile apps. We make our apps available on Google Play and in the App Store. We only build apps that are free to download and install, and our main source of revenue consists of in-app ads. This means that the number of users of our apps determines our revenue for any given app — the more users who see and engage with the ads, the better. 


*Goal*: The goal here is to  analyze data to help our developers understand what type of apps are likely to attract more users.

To do this, we'll need to collect and analyze data about mobile apps available on Google Play and the App Store. As of September 2018, there were approximately 2 million iOS apps available on the App Store, and 2.1 million Android apps on Google Play. This is shown in the figure below from [Statistica](https://www.statista.com/).

![](App_Usage.png)

We use lists here, as we have not learnt about dataframes yet. This would be far easier with dataframes, but using lists will build character. 




## Opening the files
Includes opening and reading the files, adding them into a list and seperating the header (list element 0) and the body of the data. 

In [1]:
from csv import reader

#Opening and reading the Apple and google app datasets
opened_file_apple = open('AppleStore.csv', encoding = 'utf-8')
opened_file_google = open('googleplaystore.csv', encoding = 'utf-8')

read_file_apple = reader(opened_file_apple)
read_file_google = reader(opened_file_google)

apple_apps_data = list(read_file_apple)
apple_data_header = apple_apps_data [0] #get header
iphone =  apple_apps_data[1:] #get the rest without header

google_apps_data = list(read_file_google)
google_data_header = google_apps_data [0]
android =  google_apps_data[1:]


## Writing a function to explore the data

In [3]:
# function definition to explore data and slice it up
def explore_data(dataset, start, end, rows_and_columns=False):
    dataset_slice = dataset[start:end]    
    for row in dataset_slice:
        print(row)
        print('\n') # adds a new (empty) line after each row

    if rows_and_columns:
        print('Number of rows:', len(dataset))
        print('Number of columns:', len(dataset[0]))

In [5]:
#Printing samples of the data.

explore_data(iphone,0,1,rows_and_columns=True)
print('\n')
explore_data(android,0,1,rows_and_columns=True)

['284882215', 'Facebook', '389879808', 'USD', '0.0', '2974676', '212', '3.5', '3.5', '95.0', '4+', 'Social Networking', '37', '1', '29', '1']


Number of rows: 7197
Number of columns: 16


['Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN', '4.1', '159', '19M', '10,000+', 'Free', '0', 'Everyone', 'Art & Design', 'January 7, 2018', '1.0.0', '4.0.3 and up']


Number of rows: 10841
Number of columns: 13


More information about the columns in the data sets are available below:
For apple dtaa set: https://www.kaggle.com/ramamet4/app-store-apple-data-set-10k-apps
for google data set: https://www.kaggle.com/lava18/google-play-store-apps

## Data cleaning:
We need to clean the data, by this we mean corect or remove the worng data
remove duplicates and since we are focusing on english and free apps, to remove non engish and paid apps. to make the data set fit for purpose.

From the discussion here:: https://www.kaggle.com/lava18/google-play-store-apps/discussion/66015 we can see that data entry for index 10472 is missing the category column and a coloumn shift has happened for that index. so we remove that.

## Step 1: removing wrong data

In [6]:
#removing errouneous row 10472 form the google data set
del android[10472]

## Step 2: Removing duplicates

I have done it for iphone data set too as two rows are duplicates in there but the majority of duplicates is for apple

**Part 1:**

The google data set has duplicate entries. Here are some examples of duplicate entries in this data set below: 

In [5]:
for app in android:
    name = app[0]
    if name == 'Google My Business':
        print (app)

['Google My Business', 'BUSINESS', '4.4', '70991', 'Varies with device', '5,000,000+', 'Free', '0', 'Everyone', 'Business', 'July 24, 2018', '2.19.0.204537701', '4.4 and up']
['Google My Business', 'BUSINESS', '4.4', '70991', 'Varies with device', '5,000,000+', 'Free', '0', 'Everyone', 'Business', 'July 24, 2018', '2.19.0.204537701', '4.4 and up']
['Google My Business', 'BUSINESS', '4.4', '70991', 'Varies with device', '5,000,000+', 'Free', '0', 'Everyone', 'Business', 'July 24, 2018', '2.19.0.204537701', '4.4 and up']


You can see google my business has 3 entires. Sometimes they are different because of the different number of reviews.



In [8]:
#counting the number of duplicates - for loops and appendingto lists, similar to frequency tables
duplicate_apps = []
unique_apps = []
#for android
for app in android:
    name = app[0]
    if name in unique_apps:
        duplicate_apps.append(name)
    else:
        unique_apps.append(name)
print('Number of duplicate apps android:', len(duplicate_apps))

#for iphone
duplicate_apps = []
unique_apps = []

for app in iphone:
    name = app[1]
    if name in unique_apps:
        duplicate_apps.append(name)
    else:
        unique_apps.append(name)
print('Number of duplicate apps iphone:', len(duplicate_apps))

Number of duplicate apps android: 1181
Number of duplicate apps iphone: 2


**Part 2**
We use the number of user ratings as the criterion for deleting the duplicate data and selecting the row we keep. This is because thats the only accesible criterion for both iphone and android (iphone does not have the number of downloads etc,) it has user rating count for all versions.

In [14]:
#creating the dictionary to remove duplicates (keeping highest number of rating)

#for android

reviews_max = {} 
for app in android: #loop through the list
    name = app[0] #element at index 0 is the name
    n_reviews = float(app[3]) #element at index 3 is the number of reviews
    if name in reviews_max and reviews_max[name] < n_reviews: #if we have seen it beofre compare the reviews and pick the larger one
        reviews_max[name] = n_reviews
    if name not in reviews_max: # if we have not seen the name before, then keep it and its number of reviwes. 
        reviews_max [name] = n_reviews
        
#using dictonary to remove duplicates 
android_clean = []
already_added = []

for app in android:
    name = app[0]
    n_reviews = float(app[3])
    if (n_reviews == reviews_max[name]) and (name not in already_added):
        android_clean.append (app)
        already_added.append(name)
    
print("The number of unique android entires is:", len(android_clean))

###____________________________________
#for iphone

reviews_max = {} 
for app in iphone:
    name = app[1] #track name is index 1 in the apple data base
    n_reviews = float(app[5])
    if name in reviews_max and reviews_max[name] < n_reviews:
        reviews_max[name] = n_reviews
    if name not in reviews_max:
        reviews_max [name] = n_reviews
        
#using dictonary to remove duplicates 
iphone_clean = []
already_added = []

for app in iphone:
    name = app[1]
    n_reviews = float(app[5])
    if (n_reviews == reviews_max[name]) and (name not in already_added):
        iphone_clean.append (app)
        already_added.append(name)
    
print("The number of unique iphone entires is:", len(iphone_clean))

The number of unique android entires is: 9659
The number of unique iphone entires is: 7195


## Removing Non-English Apps

**Part 1** the function

In [16]:
#function to find English app names, returns false if more than 3 non-english characters
def isenglish (input_string): 
    counter = 0
    
    for character in input_string:
        if ord(character) > 127:
            counter += 1
            
    if counter > 3:
        return False
    else:
        return True
        
print(isenglish('Docs To Go™ Free Office Suite'))
print(isenglish('Instachat 😜'))

## I had a mistake in this function, I got the indents wrong, the if statement for the 
## counter vairable was *inside* the loop. Python is really finiky with idents!! be careful!


True
True


**Part2:** the execution

In [18]:
#for the Android data set English test
android_English = []
for app in android_clean:
    name = app[0]
    if isenglish(name):
        android_English.append(app)

        
#for the iphone apps taking only English app names        
iphone_English =[]      
for app in iphone_clean:
    name = app[1]
    if isenglish(name):
        iphone_English.append(app)
        
#exploring data post non-English deletion:
print("The dimentions for the English unique android apps are")
explore_data(android_English, 0, 0, True)
print('\n')
print("The dimentions for the English unique iphone apps are")
explore_data(iphone_English, 0, 0, True)
    

The dimentions for the English unique android apps are
Number of rows: 9614
Number of columns: 13


The dimentions for the English unique iphone apps are
Number of rows: 6181
Number of columns: 16


## Finding the Free Apps

In [21]:
#isolating the free apps in each data set
android_free = [] #for android
#for android:
for app in android_English:
    app_type =  app[6]
    if app_type =='Free':
        android_free.append(app)
print(f"There are {len(android_free)} free apps for andrid")        
        
#for android:
iphone_free = []
for app in iphone_English:
    cost = float (app[4])
    if cost ==0:
        iphone_free.append(app)
print(f"There are {len(iphone_free)} free apps for iphone")  

There are 8863 free apps for andrid
There are 3220 free apps for iphone


## Most comon Apps by Genre - 
**Slide 9**
Our aim is to determine the kinds of apps that are likely to attract more users because our revenue is highly influenced by the number of people using our apps.

So far, we've spent a good amount of time cleaning data, including the following:

1. Removing inaccurate data
2. Removing duplicate app entries
3. Removing non-English apps
4. Isolating the free apps

As we mentioned in the introduction, our goal is to determine the kinds of apps that are likely to attract more users because the number of people using our apps affect our revenue.

To minimize risks and overhead, our validation strategy for an app idea has three steps:

- Build a minimal Android version of the app, and add it to Google Play.
- If the app has a good response from users, we develop it further.
- If the app is profitable after six months, we build an iOS version of the app and add it to the App Store.

Because our end goal is to add the app on both Google Play and the App Store, we need to find app profiles that are successful in both markets. For instance, a profile that works well for both markets might be a productivity app that makes use of gamification.

Let's begin the analysis by determining the most common genres for each market. For this, we'll need to build frequency tables for a few columns in our datasets.

In [24]:
#Slide 10

# Building the frequency table function, index is the column number, returns ft{}
def freq_table(dataset, index):
    ft = {} # empty frequency table
    ft_percent = {} # empty frequency table percentages
    total = 0
    
    for element in dataset: # Discarded the header for both datasets already
        total += 1
        if element[index] in ft:
            ft[element[index]] += 1
        else:
            ft[element[index]] = 1
     
    for key in ft:
        percentage = (ft[key]/total)*100
        ft_percent[key] = percentage
        
    return ft_percent

#using our function in the display and sorter function from the course
def display_table(dataset, index):
    table = freq_table(dataset, index)
    table_display = []
    for key in table:
        key_val_as_tuple = (table[key], key)
        table_display.append(key_val_as_tuple)

    table_sorted = sorted(table_display, reverse = True)
    for entry in table_sorted:
        print(entry[1], ':', entry[0])
        
        
#trying the function for frequency table of the columns prime_genre, Genres, and Category

display_table(iphone_free, 11) #iphone dataset is iphone_free and the "prime genre" column index is 10
print('\n')
display_table(android_free, 1) #Category is index 1
print('\n')
display_table(android_free, 9) #Genres is index 9 for this dataset


Games : 58.13664596273293
Entertainment : 7.888198757763975
Photo & Video : 4.968944099378882
Education : 3.6645962732919255
Social Networking : 3.291925465838509
Shopping : 2.608695652173913
Utilities : 2.515527950310559
Sports : 2.142857142857143
Music : 2.049689440993789
Health & Fitness : 2.018633540372671
Productivity : 1.7391304347826086
Lifestyle : 1.5838509316770186
News : 1.3354037267080745
Travel : 1.2422360248447204
Finance : 1.1180124223602486
Weather : 0.8695652173913043
Food & Drink : 0.8074534161490683
Reference : 0.5590062111801243
Business : 0.5279503105590062
Book : 0.43478260869565216
Navigation : 0.18633540372670807
Medical : 0.18633540372670807
Catalogs : 0.12422360248447205


FAMILY : 18.898792733837304
GAME : 9.725826469592688
TOOLS : 8.462146000225657
BUSINESS : 4.592124562789123
LIFESTYLE : 3.9038700214374367
PRODUCTIVITY : 3.8925871601038025
FINANCE : 3.7007785174320205
MEDICAL : 3.5315355974275078
SPORTS : 3.396141261423897
PERSONALIZATION : 3.317161232088458

**Conclusions from the above data**

App store: apple is mostly biased towards games and entertainment

Android is more balanced there are not that many apps designed for fun, and it seems that a good number of apps are designed for practical purposes (family, tools, business, lifestyle, productivity, etc.). However, if we investigate this further, we can see that the family category (which accounts for almost 19% of the apps) means mostly games for kids.

## Most Popular Apps by Genre on the App Store

Below, we calculate the average number of user ratings per app genre on the App Store

In [25]:
#generating Frequency table for prime_genre
prime_genre_freq = freq_table (iphone_free,11) #prime genre

#print(prime_genre_freq)

for genre in prime_genre_freq:
    total = 0
    len_genre = 0
    for row in iphone_free:
        genre_app = row[11]
        if genre_app == genre:
            total += float(row[5]) #row 5 is the number of user ratings
            len_genre +=1
    average_ratings = total/len_genre
    print (genre,average_ratings)
        

Social Networking 71548.34905660378
Photo & Video 28441.54375
Games 22812.92467948718
Music 57326.530303030304
Reference 74942.11111111111
Health & Fitness 23298.015384615384
Weather 52279.892857142855
Utilities 18684.456790123455
Travel 28243.8
Shopping 26919.690476190477
News 21248.023255813954
Navigation 86090.33333333333
Lifestyle 16485.764705882353
Entertainment 14029.830708661417
Food & Drink 33333.92307692308
Sports 23008.898550724636
Book 39758.5
Finance 31467.944444444445
Education 7003.983050847458
Productivity 21028.410714285714
Business 7491.117647058823
Catalogs 4004.0
Medical 612.0


Navigation apps followed by Social Netwroking on avergae are the most common genre based on the number of ratings. But this could be skwed by the sheer number of user ratings for these very popular apps and it does not necessarily answer our question. 

In [27]:
#Zooming in on navigation appas
for app in iphone_free:
    if app[-5] == 'Navigation':
        print(app[1], ':', app[5]) # print name and number of ratings

Waze - GPS Navigation, Maps & Real-time Traffic : 345046
Google Maps - Navigation & Transit : 154911
Geocaching® : 12811
CoPilot GPS – Car Navigation & Offline Maps : 3582
ImmobilienScout24: Real Estate Search in Germany : 187
Railway Route Search : 5


In [28]:
#Zooming in on reference appas
for app in iphone_free:
    if app[-5] == 'Reference':
        print(app[1], ':', app[5]) # print name and number of ratings

Bible : 985920
Dictionary.com Dictionary & Thesaurus : 200047
Dictionary.com Dictionary & Thesaurus for iPad : 54175
Google Translate : 26786
Muslim Pro: Ramadan 2017 Prayer Times, Azan, Quran : 18418
New Furniture Mods - Pocket Wiki & Game Tools for Minecraft PC Edition : 17588
Merriam-Webster Dictionary : 16849
Night Sky : 12122
City Maps for Minecraft PE - The Best Maps for Minecraft Pocket Edition (MCPE) : 8535
LUCKY BLOCK MOD ™ for Minecraft PC Edition - The Best Pocket Wiki & Mods Installer Tools : 4693
GUNS MODS for Minecraft PC Edition - Mods Tools : 1497
Guides for Pokémon GO - Pokemon GO News and Cheats : 826
WWDC : 762
Horror Maps for Minecraft PE - Download The Scariest Maps for Minecraft Pocket Edition (MCPE) Free : 718
VPN Express : 14
Real Bike Traffic Rider Virtual Reality Glasses : 8
教えて!goo : 0
Jishokun-Japanese English Dictionary & Translator : 0


## Most Popular Apps by Genre on Google Play


In [16]:
display_table(android_free, 5) # the Installs columns

1,000,000+ : 15.728308699086089
100,000+ : 11.55365000564143
10,000,000+ : 10.549475346947986
10,000+ : 10.199706645605326
1,000+ : 8.394448832223853
100+ : 6.916393997517771
5,000,000+ : 6.826131106848697
500,000+ : 5.562450637481666
50,000+ : 4.772650344127271
5,000+ : 4.513144533453684
10+ : 3.542818458761142
500+ : 3.2494640640866526
50,000,000+ : 2.3017037120613786
100,000,000+ : 2.1324607920568655
50+ : 1.9180864267178157
5+ : 0.7898002933543946
1+ : 0.5077287600135394
500,000,000+ : 0.270788672007221
1,000,000,000+ : 0.2256572266726842
0+ : 0.045131445334536835


We're going to leave the numbers as they are, which means that we'll consider that an app with 100,000+ installs has 100,000 installs, and an app with 1,000,000+ installs has 1,000,000 installs, and so on.

In [13]:
#generating the frequency table category on google play

#generating Frequency table for prime_genre
category_freq = freq_table (android_free,1) #category is index1

#print(category_freq)

for category in category_freq:
    total = 0
    len_category = 0
    for row in android_free:
        category_app = row[1]
        if category_app == category:
            n_installs = row[5].replace('+','') #row 5 is the number of installs, whichwe save as float
            n_installs = n_installs.replace(',','')
            total += float(n_installs) 
            len_category +=1
    average_installs = total/len_category
    print (category,average_installs)
        

EDUCATION 1833495.145631068
SPORTS 3638640.1428571427
AUTO_AND_VEHICLES 647317.8170731707
HOUSE_AND_HOME 1331540.5616438356
VIDEO_PLAYERS 24727872.452830188
BOOKS_AND_REFERENCE 8767811.894736841
BEAUTY 513151.88679245283
ART_AND_DESIGN 1986335.0877192982
ENTERTAINMENT 11640705.88235294
PERSONALIZATION 5201482.6122448975
SOCIAL 23253652.127118643
MEDICAL 120550.61980830671
SHOPPING 7036877.311557789
TOOLS 10801391.298666667
LIBRARIES_AND_DEMO 638503.734939759
TRAVEL_AND_LOCAL 13984077.710144928
EVENTS 253542.22222222222
COMMUNICATION 38456119.167247385
PRODUCTIVITY 16787331.344927534
MAPS_AND_NAVIGATION 4056941.7741935486
WEATHER 5074486.197183099
NEWS_AND_MAGAZINES 9549178.467741935
PHOTOGRAPHY 17840110.40229885
FINANCE 1387692.475609756
LIFESTYLE 1437816.2687861272
HEALTH_AND_FITNESS 4188821.9853479853
PARENTING 542603.6206896552
DATING 854028.8303030303
FOOD_AND_DRINK 1924897.7363636363
BUSINESS 1712290.1474201474
COMICS 817657.2727272727
GAME 15588015.603248259
FAMILY 3697848.173134

In [29]:
for app in android_clean:
    if app[1] == 'COMMUNICATION' and (app[5] == '1,000,000,000+'
                                      or app[5] == '500,000,000+'
                                      or app[5] == '100,000,000+'):
        print(app[0], ':', app[5])

WhatsApp Messenger : 1,000,000,000+
imo beta free calls and text : 100,000,000+
Android Messages : 100,000,000+
Google Duo - High Quality Video Calls : 500,000,000+
Messenger – Text and Video Chat for Free : 1,000,000,000+
imo free video calls and chat : 500,000,000+
Skype - free IM & video calls : 1,000,000,000+
Who : 100,000,000+
GO SMS Pro - Messenger, Free Themes, Emoji : 100,000,000+
LINE: Free Calls & Messages : 500,000,000+
Google Chrome: Fast & Secure : 1,000,000,000+
Firefox Browser fast & private : 100,000,000+
UC Browser - Fast Download Private & Secure : 500,000,000+
Gmail : 1,000,000,000+
Hangouts : 1,000,000,000+
Messenger Lite: Free Calls & Messages : 100,000,000+
Kik : 100,000,000+
KakaoTalk: Free Calls & Text : 100,000,000+
Opera Mini - fast web browser : 100,000,000+
Opera Browser: Fast and Secure : 100,000,000+
Telegram : 100,000,000+
Truecaller: Caller ID, SMS spam blocking & Dialer : 100,000,000+
UC Browser Mini -Tiny Fast Private & Secure : 100,000,000+
Viber Mess

The communication apps are the largest by far in terms of the number of installs. This obviously shows a skew in the results.

In [30]:
under_100_m = []

for app in android_clean:
    n_installs = app[5]
    n_installs = n_installs.replace(',', '')
    n_installs = n_installs.replace('+', '')
    if (app[1] == 'COMMUNICATION') and (float(n_installs) < 100000000):
        under_100_m.append(float(n_installs))
        
sum(under_100_m) / len(under_100_m)

3257903.6493055555

The above is the average installs when we remove communication apps amove 100 million installs.

We see the same pattern for the video players category, which is the runner-up with 24,727,872 installs. The market is dominated by apps like Youtube, Google Play Movies & TV, or MX Player. The pattern is repeated for social apps (where we have giants like Facebook, Instagram, Google+, etc.), photography apps (Google Photos and other popular photo editors), or productivity apps (Microsoft Word, Dropbox, Google Calendar, Evernote, etc.).

Again, the main concern is that these app genres might seem more popular than they really are. Moreover, these niches seem to be dominated by a few giants who are hard to compete against.

The game genre seems pretty popular, but previously we found out this part of the market seems a bit saturated, so we'd like to come up with a different app recommendation if possible.

The books and reference genre looks fairly popular as well, with an average number of installs of 8,767,811. It's interesting to explore this in more depth, since we found this genre has some potential to work well on the App Store, and our aim is to recommend an app genre that shows potential for being profitable on both the App Store and Google Play.

Let's take a look at some of the apps from this genre and their number of installs:

In [31]:
for app in android_clean:
    if app[1] == 'BOOKS_AND_REFERENCE':
        print(app[0], ':', app[5])

E-Book Read - Read Book for free : 50,000+
Download free book with green book : 100,000+
Wikipedia : 10,000,000+
Cool Reader : 10,000,000+
Free Panda Radio Music : 100,000+
Book store : 1,000,000+
FBReader: Favorite Book Reader : 10,000,000+
English Grammar Complete Handbook : 500,000+
Free Books - Spirit Fanfiction and Stories : 1,000,000+
Google Play Books : 1,000,000,000+
AlReader -any text book reader : 5,000,000+
Offline English Dictionary : 100,000+
Offline: English to Tagalog Dictionary : 500,000+
FamilySearch Tree : 1,000,000+
Cloud of Books : 1,000,000+
Recipes of Prophetic Medicine for free : 500,000+
ReadEra – free ebook reader : 1,000,000+
Anonymous caller detection : 10,000+
Ebook Reader : 5,000,000+
Litnet - E-books : 100,000+
Read books online : 5,000,000+
English to Urdu Dictionary : 500,000+
eBoox: book reader fb2 epub zip : 1,000,000+
English Persian Dictionary : 500,000+
Flybook : 500,000+
All Maths Formulas : 1,000,000+
Ancestry : 5,000,000+
HTC Help : 10,000,000+
E

In [33]:
for app in android_clean:
    if app[1] == 'BOOKS_AND_REFERENCE' and (app[5] == '1,000,000+'
                                            or app[5] == '5,000,000+'
                                            or app[5] == '10,000,000+'
                                            or app[5] == '50,000,000+'):
        print(app[0], ':', app[5])

Wikipedia : 10,000,000+
Cool Reader : 10,000,000+
Book store : 1,000,000+
FBReader: Favorite Book Reader : 10,000,000+
Free Books - Spirit Fanfiction and Stories : 1,000,000+
AlReader -any text book reader : 5,000,000+
FamilySearch Tree : 1,000,000+
Cloud of Books : 1,000,000+
ReadEra – free ebook reader : 1,000,000+
Ebook Reader : 5,000,000+
Read books online : 5,000,000+
eBoox: book reader fb2 epub zip : 1,000,000+
All Maths Formulas : 1,000,000+
Ancestry : 5,000,000+
HTC Help : 10,000,000+
Moon+ Reader : 10,000,000+
English-Myanmar Dictionary : 1,000,000+
Golden Dictionary (EN-AR) : 1,000,000+
All Language Translator Free : 1,000,000+
Aldiko Book Reader : 10,000,000+
Dictionary - WordWeb : 5,000,000+
50000 Free eBooks & Free AudioBooks : 5,000,000+
Al-Quran (Free) : 10,000,000+
Al Quran Indonesia : 10,000,000+
Al'Quran Bahasa Indonesia : 10,000,000+
Al Quran Al karim : 1,000,000+
Al Quran : EAlim - Translations & MP3 Offline : 5,000,000+
Koran Read &MP3 30 Juz Offline : 1,000,000+
H

This niche seems to be dominated by software for processing and reading ebooks, as well as various collections of libraries and dictionaries, so it's probably not a good idea to build similar apps since there'll be some significant competition.

We also notice there are quite a few apps built around the book Quran, which suggests that building an app around a popular book can be profitable. It seems that taking a popular book (perhaps a more recent book) and turning it into an app could be profitable for both the Google Play and the App Store markets.

However, it looks like the market is already full of libraries, so we need to add some special features besides the raw version of the book. This might include daily quotes from the book, an audio version of the book, quizzes on the book, a forum where people can discuss the book, etc.

## Conclusions
In this project, we analyzed data about the App Store and Google Play mobile apps with the goal of recommending an app profile that can be profitable for both markets.

We concluded that taking a popular book (perhaps a more recent book) and turning it into an app could be profitable for both the Google Play and the App Store markets. The markets are already full of libraries, so we need to add some special features besides the raw version of the book. This might include daily quotes from the book, an audio version of the book, quizzes on the book, a forum where people can discuss the book, etc.